# Week 10 (Monday), AST 8581 / PHYS 8581 / CSCI 8581 / STAT 8581: Big Data in Astrophysics

### Michael Coughlin <cough052@umn.edu>

With contributions totally ripped off from Siddharth Mishra-Sharma (MIT) and Deep  Chatterjee (MIT)

# Where are we headed?

Foundations of Data and Probability -> Statistical frameworks (Frequentist vs Bayesian) -> Estimating underlying distributions -> Analysis of Time series (periodicity) -> Analysis of Time series (variability) -> Analysis of Time series (stochastic processes) -> Gaussian Processes -> Decision Trees / Regression -> Dimensionality Reduction  -> Principle Component Analysis -> Clustering / Density Estimation / Anomaly Detection -> Supervised Learning -> <b> Deep Learning </b> -> Introduction to Databases - SQL -> Introduction to Databases - NoSQL -> Introduction to Multiprocessing -> Introduction to GPUs -> Unit Testing

In [ ]:
# ! pip install --upgrade emcee corner pytorch-lightning tqdm nflows

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
import numpy as np
import emcee
from scipy.stats import poisson
from scipy.stats import chi2
from scipy.optimize import basinhopping
from tqdm import tqdm
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import corner

# Simulation-based (likelihood-free) inference

Simulation-based inference (SBI) is a powerful class of methods for performing inference in settings where the likelihood is computationally intractable, but simulations can be realized via forward modeling. 

In this lecture we will
- Introduce the notion of an implicit likelihood, and how to leverage it to perform inference;
- Look at a "traditional" method for likelihood-free inference, Approximate Bayesian Computation (ABC);
- Build up two common modern _neural_ SBI techniques: neural likelihood-ratio estimation (NRE) and neural posterior estimation (NPE);
- Introduce the concept of statistical coverage testing and calibration.

As examples, we will look at a simple Gaussian-signal-on-power-law-background ("bump hunt"), where the likelihood is tractable, and a more complicated example of inferring a distribution of point sources, where the likelihood is computationally intractable. We will emphasize what it means for a likelihood to be computationally intractable/challenging and where the advantages of SBI come in.

<img src=./assets/header.png alt= “” width=800>


### Likelihood-Free Inference (Simulation-Based Inference)

The problem of statistical inference in the absence of an explicit likelihood has been termed **likelihood-free inference** (though the term is somewhat of a misnomer). In reality, the goal is to estimate the intractable likelihood, so the term **simulation-based inference** is considered more appropriate.

The intractability of the likelihood represents a barrier to scientific progress, as statistical inference is a crucial component of the scientific method. Over time, scientists have developed various methods to overcome this challenge, often using insights specific to their field.

### Traditional Approaches to Inference

1. **Density Estimation**: 
   This method involves approximating the distribution of summary statistics derived from the simulator. It has been widely used in fields such as high-energy physics (e.g., the discovery of the Higgs boson in a frequentist framework). The key idea is to estimate the distribution of the summary statistics based on samples generated from the simulator.

2. **Approximate Bayesian Computation (ABC)**:
   In ABC, observed and simulated data are compared using a distance measure based on summary statistics. ABC is widely applied in population biology, computational neuroscience, and cosmology. The method works by iteratively refining the simulations based on how close the simulated data is to the observed data.

Both methods, despite their challenges, have been effective and widely used in many scientific domains.


## Recent Advances in Simulation-Based Inference

The field of simulation-based inference (SBI) has seen rapid growth and expansion in recent years. Three key developments have been driving this acceleration:

### 1. Cross-Pollination with Machine Learning
A significant exchange of ideas has occurred between researchers in simulation-based inference and those in the machine learning (ML) community. The growth and advancements in machine learning techniques have provided new tools and methodologies that are enhancing simulation-based inference, allowing for novel approaches that were previously unfeasible.

### 2. Active Learning
Active learning, the process of using knowledge gained during inference to guide further simulations, has become increasingly recognized as a powerful tool. By focusing on regions of parameter space that are most informative, active learning improves the sample efficiency of inference methods, reducing computational costs and enhancing the accuracy of the results.

### 3. Direct Integration with Simulators
Recent research has shifted away from treating simulators as black boxes. Instead, there has been a focus on developing integrations that allow inference engines to directly access and utilize the internal details of simulators. This approach enables a more efficient and detailed use of the simulation data, further improving the quality of inference.

## Simulation-Based Inference and Statistical Models

In simulation-based inference, the statistical model is defined by the simulator itself. A **simulator** is a computer program that takes a vector of parameters **θ**, samples internal states or latent variables **z_i** (which depend on previous latent variables), and produces a data vector **x** as output. The simulator represents a probabilistic program, where random samplings are involved and the program is interpreted as a statistical model.

### Key Components of a Simulator

A simulator in simulation-based inference can vary substantially depending on the context, but it typically includes the following components:

#### 1. **Parameters (θ)**:
   - These parameters describe the underlying mechanistic model that governs the system and affect the transition probabilities of the latent variables.
   - **θ** typically has a small number of components and fixed dimensionality, and it is interpretable by domain scientists. Examples include:
     - Coefficients in the Hamiltonian of a physical system.
     - Virulence and incubation rates of a pathogen.
     - Fundamental constants of nature.

#### 2. **Latent Variables (z)**:
   - These variables play a crucial role in the data-generating process but are typically unobservable in practice. The structure of the latent space can vary significantly between simulators:
     - Latent variables may be **continuous** or **discrete**, and the dimensionality may be fixed or dynamic depending on the simulator's control flow.
     - Simulators may combine **deterministic** and **stochastic** steps, with the deterministic components either being differentiable or involving discontinuous control flows.
     - In practice, some simulators offer easy access to latent variables, while others are more like "black boxes" with limited transparency.
   
#### 3. **Output Data (x)**:
   - The data produced by the simulator corresponds to the observations or outputs that are observed in the real-world scenario being modeled. These data can vary in complexity:
     - From simple unstructured numbers to high-dimensional or highly structured data, such as images, time-series data, or geospatial information.

<img src=./assets/sbi.png alt= “” width=700>

# Inference Tasks and Challenges in Simulation-Based Inference

## Types of Inference Tasks

Scientific inference tasks can vary depending on **what** is being inferred. Given observed data **x**, the goal may be to infer:
- **Input parameters (θ)**: These are the parameters that define the system's behavior.
- **Latent variables (z)**: These are variables that contribute to the data generation but are unobservable in practice.
- **Both**: Sometimes, both parameters and latent variables are of interest.

In some cases, only a subset of the parameters or latent variables may be of interest, while the rest are considered **nuisance parameters**. Nuisance parameters are not directly relevant to the inference but must be accounted for because they influence the distribution of the observed data.

For this discussion, we will focus on the common problem of inferring **θ** in a parametric setting. We will also touch on methods for inferring **z**, but will not delve into non-parametric inverse problems.

## Frequentist vs. Bayesian Inference

Inference can be approached in two main ways:
1. **Frequentist Inference**:
   - Often focuses on point estimates **θ̂(x)**, with uncertainty represented as confidence sets.
   - Confidence sets are typically derived by inverting hypothesis tests, using the likelihood ratio test statistic.

2. **Bayesian Inference**:
   - In Bayesian inference, the goal is to compute the posterior distribution:
     $
     p(\theta | x) = \frac{p(x | \theta) \, p(\theta)}{\int \! \! \! \text{d} \theta' \, p(x | \theta') \, p(\theta')}
     $
     where **p(x | θ)** is the likelihood, **p(θ)** is the prior, and the denominator represents the marginal likelihood over all possible parameter values.
   - Bayesian inference provides a probabilistic measure of uncertainty by calculating the full posterior distribution over the parameters **θ**.

In both cases, the likelihood function **p(x | θ)** plays a central role.

## The Challenge of Intractable Likelihoods

In simulation-based inference, the **likelihood function** defined by the simulator is typically intractable. The likelihood is defined as an integral over all possible trajectories through the latent space, i.e., all possible execution traces of the simulator:
\[
p(x | \theta) = \intz p(x, z | \theta)
\]
where **p(x, z | θ)** is the joint probability density of the observed data **x** and the latent variables **z**.

For simple sequential data generation processes, the joint likelihood can be expressed as:
\[
p(x, z | \theta) = p(x | \theta, z) \prod_{i} p_i(z_i | \theta, z_{<i})
\]
However, for real-life simulators with large latent spaces, it becomes computationally impossible to calculate this integral explicitly. Since the likelihood function is central to both frequentist and Bayesian inference, this intractability presents a major challenge for inference in many scientific fields.


<img src=./assets/likelihood.png alt= “” width=700>

## Classical Approaches to Inference Without Tractable Likelihoods

The problem of performing inference without a tractable likelihood function is not new. Over the years, two major approaches have been developed to address this challenge.

### 1. Approximate Bayesian Computation (ABC)

One of the most well-known approaches is **Approximate Bayesian Computation (ABC)**, which was first introduced by Rubin (1984) and Beaumont (2002). In the simplest form of rejection ABC:
- Parameters **θ** are drawn from the prior distribution.
- The simulator is run with the chosen **θ** to sample **x_sim** from the likelihood distribution **p(x | θ)**.
- The parameter **θ** is accepted as a posterior sample if the simulated data **x_sim** is sufficiently close to the observed data **x_obs**. This is determined by a distance measure **ρ(x_sim, x_obs)** and a tolerance **ε**.

In essence, the likelihood is approximated by the probability that the condition **ρ(x_sim, x_obs) < ε** holds. The accepted samples then approximate the posterior distribution.

### 2. Approximate Frequentist Computation

The second classical approach to simulation-based inference involves creating a model for the likelihood by estimating the distribution of simulated data using techniques like histograms or kernel density estimation. Frequentist and Bayesian inference then proceed as if the likelihood were tractable.

<img src=./assets/abc.png alt= “” width=700>


## Shortcomings of Traditional Simulation-Based Inference Techniques

Traditional simulation-based inference techniques, such as **Approximate Bayesian Computation (ABC)** and **classical density estimation**, have been widely used in many fields. However, they suffer from three key shortcomings:

### 1. Sample Efficiency
- **Curse of Dimensionality**: Both ABC and classical density estimation suffer from poor scaling with high-dimensional data. This means that a large number of simulated samples is required to estimate the likelihood or posterior accurately.
- The number of simulations needed to obtain reliable results can become prohibitively expensive, especially as the dimensionality of the data increases.

### 2. Quality of Inference
- **Information Loss**: Reducing the data to low-dimensional summary statistics inevitably discards some of the information about the parameters **θ**, leading to a loss in statistical power.
- In **ABC**, large values of the **ε** parameter (the tolerance for accepting samples) lead to poor approximations of the true likelihood.
- In **density estimation**, large bandwidths for kernel density estimation can also result in poor approximations.
- Both issues reduce the overall **quality of inference**, making the results less reliable.

### 3. Amortization
- **Repetition of Steps**: With ABC, each new set of observed data requires repeating most steps of the inference process, especially if the proposal distribution depends on the observed data. This makes ABC inefficient when dealing with large datasets or multiple observations.
- In contrast, **density estimation-based inference** is **amortized**, meaning the computationally expensive steps (like simulation and density estimation) are done once and can be reused for new data. This is particularly useful for problems with many i.i.d. observations, where repeated inference is required.

# Advances in Simulation-Based Inference

Recent developments have significantly improved the three major challenges of traditional simulation-based inference: **sample efficiency, quality of inference, and amortization**. These advances can be grouped into three main directions of progress:

## 1. Machine Learning Revolution
- **Higher-Dimensional Data**: Modern deep learning techniques allow inference methods to work with high-dimensional data without relying on manually chosen summary statistics.
- **Neural Network Surrogates**: Inference methods leveraging neural networks directly benefit from the rapid advancements in deep learning, improving the quality and accuracy of inference.

## 2. Active Learning for Sample Efficiency
- **Optimized Data Collection**: Active learning techniques iteratively guide the simulator to generate the most informative samples, improving sample efficiency.
- **Reduction in Computational Costs**: By focusing simulation efforts on the most relevant regions of parameter space, active learning makes it feasible to tackle computationally expensive simulators.

## 3. Integration of Differentiation and Probabilistic Programming
- **Beyond Black-Box Simulators**: Advances in **automatic differentiation** and **probabilistic programming** allow inference methods to directly tap into the internal workings of the simulator.
- **Augmenting Training Data**: Additional information extracted from the simulator (such as gradients) enhances inference, leading to more efficient and accurate probabilistic modeling.

## Expanding the Frontier of Inference
These advancements have significantly expanded the landscape of simulation-based inference, enabling researchers to handle **higher-dimensional data** and **more complex simulators** than ever before.

<img src=./assets/frontiers.png alt= “” width=500>

# Neural Density Estimation in High Dimensions

A major area of advancement in simulation-based inference is **density estimation in high-dimensional spaces**. Classical techniques such as **histograms** and **kernel density estimation** struggle with the curse of dimensionality, making them ineffective for complex datasets. Neural networks are now widely used to tackle this problem, leading to the development of **neural density estimation techniques**.

## Normalizing Flows for Density Estimation
One particularly powerful class of neural density estimation methods is **normalizing flows**. These models enable efficient and scalable density estimation through the following mechanism:

1. **Base Distribution**: A simple, known probability distribution $p(u)$ (e.g., a multivariate Gaussian) is chosen as the starting point.
2. **Invertible Transformations**: A parameterized, invertible function $x = g_\phi(u)$ is applied to transform the base distribution into a more complex target distribution.
3. **Change-of-Variables Formula**: The density of the transformed variable is computed as:
   $
   p_g(x) = p(u) \left| \det \frac{d g_\phi^{-1}(x)}{dx} \right|
   $
   where the determinant of the Jacobian accounts for the transformation's effect on volume.
4. **Stacking Transformations**: Multiple transformations can be composed in sequence, allowing the probability density to **“flow”** through different variable spaces, capturing intricate structures in the data.

## Training Normalizing Flows
- The parameters $\phi$ of the transformations are optimized by **maximizing the likelihood** of the observed data under the model, ensuring that $p_g(x)$ closely approximates the unknown true density $p(x)$.
- A key advantage of this approach is that it enables both **density estimation** (evaluating $p(x)$ and **data generation** (sampling new points by drawing $u$ from the base distribution and applying the learned transformations).

## Conditional Neural Density Estimation
- Normalizing flows and similar neural density estimators can be extended to **model conditional densities**, such as:
  - **Likelihood estimation** $p(x|\theta)$, where the model learns to approximate the probability of data given specific parameters.
  - **Posterior estimation** $p(\theta|x)$, where the model captures the distribution over parameters given observed data.
- These extensions make neural density estimators particularly useful for simulation-based inference, enabling efficient and scalable probabilistic modeling.

Neural density estimation is now a fundamental tool in high-dimensional statistical inference, opening the door to more expressive and scalable simulation-based inference techniques.


<img src=./assets/nre.png alt= “” width=700>

<img src=./assets/npe.png alt= “” width=700>

# Autoregressive Models for Density Estimation

Another powerful class of neural density estimation techniques is **autoregressive models**. These models decompose a high-dimensional probability distribution into a sequence of **conditional densities** for individual components:

$
p(x) = p(x_1) p(x_2 | x_1) p(x_3 | x_1, x_2) \dots p(x_d | x_{<d})
$

This factorization allows for **expressive** models with **tractable density evaluation** and the ability to generate synthetic data.

## Key Features of Autoregressive Models
- **Expressivity**: These models can capture complex dependencies between variables.
- **Tractable Density**: The sequential decomposition ensures that each conditional probability can be explicitly computed.
- **Synthetic Data Generation**: New samples can be drawn step-by-step, conditioning each step on the previously generated values.

## Trade-offs and Applications
- **Computational Cost**: Unlike some other neural density estimators, autoregressive models can be **slower** for sample generation since variables must be sampled sequentially.
- **Alignment with Simulators**: Many scientific simulators naturally follow a sequential data generation process. This **structural similarity** between simulators and autoregressive models presents an opportunity to align **neural network latent variables** with the **semantically meaningful latent variables** of a simulator.

Despite their computational drawbacks in industrial applications, autoregressive models offer a promising avenue for **simulation-based inference**, particularly when the latent variable structure of a simulator can be leveraged.


# Generative Adversarial Networks (GANs) in Simulation-Based Inference

**Generative Adversarial Networks (GANs)** offer an alternative approach to neural network-based generative modeling. Unlike normalizing flows or autoregressive models, GANs do not impose invertibility constraints on the transformation implemented by the generator. This increased **expressiveness** comes at the cost of an **intractable density function**, making maximum likelihood training infeasible.

## How GANs Work
GANs consist of two competing neural networks:
- **Generator (G):** Learns to map random noise to realistic samples.
- **Discriminator (D):** Attempts to distinguish between real samples and those generated by \( G \).

The two networks are trained in a **minimax game**, where \( G \) tries to fool \( D \), and \( D \) tries to correctly classify real vs. generated data.

## Implications for Simulation-Based Inference
- **No Explicit Density Estimation**: Unlike normalizing flows or autoregressive models, GANs do not provide a tractable density function \( p(x) \).
- **Flexible and High-Quality Generations**: The lack of invertibility constraints allows the generator to learn complex transformations, making GANs particularly powerful for **high-dimensional** or **structured** data.
- **The Likelihood Ratio Trick**: A key insight in simulation-based inference is that the adversarial setup of GANs can be adapted to estimate likelihood ratios, enabling inference methods without directly computing the likelihood.

# Workflows for Simulation-Based Inference

## Overview
Simulation-based inference methods vary in structure, with some closely resembling traditional approaches like Approximate Bayesian Computation (ABC) and density estimation, while others introduce fundamentally new workflows. To navigate this diverse landscape, we first outline common **building blocks** before assembling them into specific inference algorithms.

## Common Components of Inference Workflows
### 1. Running the Simulator
- The simulator plays a central role in all inference methods (represented as a **yellow pentagon** in the next figure).
- The parameters for simulation are drawn from a **proposal distribution**, which may:
  - Be static or adaptively updated (e.g., using **active learning**).
  - Depend on the prior in Bayesian settings.

### 2. Data Processing
- The high-dimensional simulator output can be:
  - **Used directly** for inference.
  - **Reduced to summary statistics**, which may be:
    - **Prescribed** by experts.
    - **Learned** from data.

## Broad Categories of Inference Methods
Inference methods can be divided into two main approaches:

1. **Simulator-Dependent Methods (e.g., ABC)**  
   - The simulator is directly used during inference.  
   - The simulator output is compared to observed data (top panels of the next figure).

2. **Surrogate-Based Methods**  
   - The simulator generates training data for a separate estimation or machine learning model (green boxes in the figure).  
   - A trained surrogate model (**red hexagons**) is used for inference.

## Addressing Intractability of the Likelihood
Different algorithms handle the challenge of an intractable likelihood function in various ways:
- **Likelihood Estimation**: Construct a tractable surrogate for the likelihood.
- **Likelihood Ratio Estimation**: Estimate the likelihood ratio function, facilitating frequentist inference.
- **Implicit Inference**: Avoid explicit likelihood calculations, such as in rejection-based ABC methods (which do not naturally support frequentist inference).

## Posterior Estimation in Bayesian Inference
The ultimate goal in Bayesian inference is the posterior distribution. Different methods provide:
- **Posterior samples** (e.g., from **MCMC or ABC**).
- **Tractable approximations** of the posterior function.

Additionally, workflows differ in **when** inference targets are specified:
- Some require defining target quantities **early** in the process.
- Others allow for flexibility, postponing this decision until later.


<img src=./assets/workflows10.png alt= “” width=700>

# Amortized Inference and Surrogate Modeling

## The Need for Amortization
One key drawback of using the simulator directly during inference is the **lack of amortization**:
- Each time new observed data becomes available, the **entire inference chain must be repeated**.
- This makes direct simulation-based inference computationally inefficient, particularly for large datasets.

By contrast, training a **surrogate model** or **emulator** for the simulator enables **amortized inference**:
- After an **initially expensive** simulation and training phase, inference on new data becomes highly efficient.
- This approach is particularly effective for **datasets with many i.i.d. observations**.

## Inverting the Simulator: A Direct Approach
A straightforward approach is to **invert the simulator**:
- Train a model to predict the **true parameters** $\hat{\theta}(x)$ from observed data $(x\).

### Limitations of Direct Parameter Estimation
Despite its simplicity, direct parameter estimation has drawbacks:
- **Lack of probabilistic interpretation**: These methods provide **point estimates**, rather than **uncertainty estimates**.
- **No likelihood or posterior modeling**: This limits their utility in Bayesian inference and confidence estimation.

For these reasons, the focus is often on **learning probabilistic models** of the likelihood, likelihood ratio, or posterior, rather than direct inversion.

---

This discussion provides the foundation for various amortized inference workflows, illustrated in the **bottom panels of the above figure**.


# Surrogate Models for Simulation-Based Inference

## Probabilistic Density Estimation
A powerful approach to amortized inference is training **neural conditional density estimators** as **surrogates for the simulator**. These networks can approximate the conditional density in two ways:
- **Posterior estimation**: Learn $p(\theta | x)$ directly.
- **Likelihood estimation**: Learn $p(x | \theta)$ as a surrogate for the simulator.

These techniques are illustrated in panel e and f of the above figure. Notably, the likelihood-based approach **resembles classical density estimation methods** but leverages more advanced machine learning techniques.

## Learning the Likelihood Ratio
An alternative to modeling densities directly is to train a neural network to estimate the **likelihood ratio**:
- $\frac{p(x|\theta_0)}{p(x|\theta_1)}$ (comparison of two parameter values).
- $\frac{p(x|\theta_0)}{p(x)}$, where $p(x)$ is integrated over a proposal or prior.

This approach (shown in panel g of the figure) is closely related to **GAN-based discriminators**:
- A classifier is trained to **discriminate between data generated at two different parameter values $\theta_0$ and $\theta_1$**.
- By leveraging the **Neyman-Pearson lemma**, the classifier output can be **converted into an estimate of the likelihood ratio**—a technique known as the **likelihood ratio trick**.

## Advantages of Surrogate-Based Approaches
These methods share key advantages:
1. **Amortized inference**: After training, inference can be performed efficiently for **arbitrary data and parameters**.
2. **No need for manual summary statistics**: The neural network learns the relevant structures in **high-dimensional data**.
3. **Active learning potential**: By iteratively refining the proposal distribution, simulations can be **guided toward relevant parameter regions**, improving sample efficiency.

## Choosing Between Likelihood, Likelihood Ratio, and Posterior Learning
Despite their similarities, these approaches differ in key ways:

| Method          | Bayesian or Frequentist? | Key Advantages | Key Drawbacks |
|---------------|---------------------|-----------------|---------------|
| **Posterior Estimation $p(\theta | x)$** | Bayesian | Directly learns the target posterior | Prior-dependent at all inference stages |
| **Likelihood Estimation $p(x | \theta)$** | Both | Enables **frequentist inference** and **Bayesian inference** (with MCMC/VI) | Requires an additional step to generate posterior samples |
| **Likelihood Ratio Estimation $\frac{p(x | \theta_0)}{p(x | \theta_1)}$** | Both | More **flexible** (prior-independent), can be trained via **supervised learning** | Does not produce generative samples directly |

## Supervised vs. Unsupervised Learning
- **Likelihood and posterior estimation** are **unsupervised learning problems**.
- **Likelihood ratio estimation** is a **supervised learning problem** (training a classifier), often making it a simpler and more efficient task.

Since **likelihood and likelihood ratio methods are interchangeable for inference**, **learning the likelihood ratio function is often computationally advantageous**.


## Affine Autoregressive Transforms

An **Affine Autoregressive Transform** is a type of **autoregressive flow** used in normalizing flows for **density estimation and generative modeling**. It defines a **learnable, invertible transformation** of a random variable where each dimension is transformed **sequentially** using an affine function whose parameters depend on the previously transformed dimensions.

### Breaking It Down

1. **Affine Transformation**  
   Each component of the transformed variable is computed as:  
   $
   x_i = s_i(z_{1:i-1}) \cdot z_i + t_i(z_{1:i-1})
   $
   where:
   - $ x_i $ is the transformed variable,
   - $ z_i $ is the input variable,
   - $ s_i $ (scaling factor) and $ t_i $ (translation factor) are functions of the **previous** components $ z_{1:i-1} $,
   - The transformation is **invertible** as long as $ s_i \neq 0 $.

2. **Autoregressive Structure**  
   - The transformation is **sequential**: each output dimension $ x_i $ depends only on **previous** inputs $ z_{1:i-1} $.
   - This factorization makes it possible to model complex **high-dimensional distributions**.

3. **Use in Normalizing Flows**  
   - Autoregressive flows allow tractable **density evaluation** via the change of variables formula:
     $
     p(x) = p(z) \left| \det J \right|^{-1}
     $
     where $ J $ is the Jacobian of the transformation.
   - Since the Jacobian is **triangular**, its determinant is just the **product of scaling factors** $ s_i $, making density computation **efficient**.

### Examples of Affine Autoregressive Transforms
- **MAF (Masked Autoregressive Flow)**: Uses deep neural networks to parameterize $ s_i $ and $ t_i $, ensuring the autoregressive property.
- **IAF (Inverse Autoregressive Flow)**: Uses a similar structure but in **inverse mode**, making it fast for **sampling** rather than density evaluation.
- **RealNVP (Real-Valued Non-Volume Preserving Flow)**: A simplified version using coupling layers, where only half of the dimensions depend on the others.


# Further reading

- [The frontier of simulation-based inference](https://arxiv.org/abs/1911.01429) (Cranmer, Brehmer, Louppe): Review paper
- [simulation-based-inference.org](http://simulation-based-inference.org/): List of papers and resources
- [awesome-neural-sbi](https://github.com/smsharma/awesome-neural-sbi): List of papers and resources

# In-class warm-up: Learning a multimodal distribution

Affine Autoregressive Transforms to learn the transform from a standard normal into a two-moon distribution. The code is light and can be run on a local laptop; no GPUs needed.

- Transforms are based on `pyro`, which closely embraces the `torch.distributions` library.
- `sklearn` two-moon dataset is used.
- `matplotlib` for plotting

In [1]:
# !pip install pyro sklearn matplotlib

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
from sklearn import datasets

import torch
from torch import distributions as dist
from torch import optim

from pyro.nn import AutoRegressiveNN
from pyro.distributions import TransformedDistribution
from pyro.distributions.transforms import AffineAutoregressive

In [ ]:
samples, labels = datasets.make_moons(n_samples=1000, noise=0.1)

In [ ]:
plt.scatter(samples.T[0], samples.T[1])
plt.title("Two moon distribution")
plt.xlabel("$x$")
plt.ylabel("$y$")

In [ ]:
samples = torch.from_numpy(samples).to(dtype=torch.float32)

## Autoregressive Net and Transform

The flow we implement below has affine autoregressive transforms. Most of the constructs are available in the `pyro` API.

In [ ]:
input_dim = 2  # data dimension
hidden_dims = [50*input_dim, 50*input_dim, 50*input_dim]

base_dist = dist.Normal(torch.zeros(input_dim), torch.ones(input_dim))

arn = AutoRegressiveNN(input_dim, hidden_dims, param_dims=[1, 1])

In [ ]:
arn

In [ ]:
transform =  AffineAutoregressive(arn)  # the "affine" part implies the linear relation between hidden dimensions

In [ ]:
# the flow implementation is torch transformed distribution
flow_dist = dist.TransformedDistribution(base_dist, [transform])

In [ ]:
optimizer = optim.Adam(transform.parameters(), lr=1e-4)

In [ ]:
from IPython.display import clear_output
from time import sleep

def live_plot(x_vals, y_vals, iteration):
    """Auxiliary function to visualize the distribution"""
    clear_output(wait=True)
    sleep(1)
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.scatter(x_vals, y_vals, label='proxy')
    ax.scatter(samples.T[0], samples.T[1], alpha=0.1, label='Orig.')
    ax.legend()
    ax.set_title('iteration {}'.format(iteration))

## Learn the Transform

In [ ]:
num_iter = 1000
for i in range(num_iter):

    optimizer.zero_grad()
    # take the original samples, and evaluate the likelihood.
    loss = -flow_dist.log_prob(samples).mean()
    loss.backward()
    optimizer.step()

    flow_dist.clear_cache()  # pyro modules cache values and derivatives for performance

    if (i + 1) % 100 == 0:
        with torch.no_grad():
            samples_flow = flow_dist.sample(torch.Size([1000,])).numpy()
        live_plot(samples_flow[:,0], samples_flow[:,1], i + 1)
        plt.show()

## Compose several transforms

In the previous case we just had a single transform

In [ ]:
transforms = [
    AffineAutoregressive(
        AutoRegressiveNN(input_dim, hidden_dims,
                         param_dims=[1, 1])
    ) for _ in range(5)
]

In [ ]:
flow_dist = dist.TransformedDistribution(base_dist, transforms)

In [ ]:
trainable_parameters = []

for t in transforms:
    trainable_parameters.extend(list(t.parameters()))

In [ ]:
optimizer = optim.Adam(trainable_parameters, lr=1e-4)

### Learn the transform

In [ ]:
num_iter = 1000
for i in range(num_iter):

    optimizer.zero_grad()
    loss = -flow_dist.log_prob(samples).mean()
    loss.backward()
    optimizer.step()
    flow_dist.clear_cache()

    if (i + 1) % 100 == 0:
        with torch.no_grad():
            samples_flow = flow_dist.sample(torch.Size([1000,])).numpy()

        live_plot(samples_flow[:,0], samples_flow[:,1], i + 1)
        plt.show()

In [ ]:
num_parameters = lambda parameters: sum(p.numel() for p in parameters if p.requires_grad)

In [ ]:
print("Trainable parameters of single transform =", num_parameters(transform.parameters()))

In [ ]:
print("Trainable parameters of after composing transforms =", num_parameters(trainable_parameters))

# Things to try

- Compare results/number of trainable parameters from other flavors of autoregressive nets: splines, neural autoregressive etc.
- Compare results/number of parameters with coupling layers instead. Note that like affine/spline autoregressive, there is the corresponding affine/spline coupling transforms.

# Other exercises

Depending on whether the masked feed-forward layers are implemented from "data" to "normal" direction or opposite, the flow is called masked-autoregressive or inverse-autoregressive. Look at the `pyro` source code on github and infer which one is the above implementation.

## When the distribution is easy to evaluate

In the exercise above we had "samples" from the the two-moon distribution, but did not have access to the true distribution. Consider the other regime, where we can evaluate the true distribution i.e. don't have samples. Train a flow to learn the following distribution.

In [ ]:
def multimodal_pdf(x, y):
    res = np.exp(
        - x**2 - (8 + 4*x**2 + 8*y)**2
    )
    res += 0.5*np.exp(
        - 8*x**2 - 8*(y - 2)**2
    )
    res *= 16./3./np.pi
    return np.log(res)

In [ ]:
x_vals = np.linspace(-3, 3, 100)
y_vals = np.linspace(-3, 3, 100)

X, Y = np.meshgrid(x_vals, y_vals)

Z = multimodal_pdf(X, Y)

fig, ax = plt.subplots(figsize=(6, 5))
cs = ax.contourf(X, Y, np.exp(Z), levels=5)
cbar = fig.colorbar(cs)

ax.set_title("$p^{*}(x, y)$")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")

# Simple bump-on-power-law example

As an initial example, consider a Gaussian signal parameterized by {amplitude, mean location, std} on top of a power law background parameterize by {amplitude, power-law exponent}.
$$ x_b = A_b\,y^{n_b}$$
$$x_s = A_s\,\exp^{-(y - \mu_s)^2 / 2\sigma_s^2}$$
$$x \sim \mathrm{Pois}(x_b + x_s)$$

In [ ]:
def bump_forward_model(y, amp_s, mu_s, std_s, amp_b, exp_b):
    """ Forward model for a Gaussian bump (amp_s, mu_s, std_s) on top of a power-law background (amp_b, exp_b).
    """
    x_b = amp_b * (y ** exp_b)  # Power-law background
    x_s = amp_s * np.exp(-((y - mu_s) ** 2) / (2 * std_s ** 2))  # Gaussian signal

    x = x_b + x_s  # Total mean signal

    return x

In [ ]:
def poisson_interval(k, alpha=0.32): 
    """ Uses chi2 to get the poisson interval.
    """
    a = alpha
    low, high = (chi2.ppf(a/2, 2*k) / 2, chi2.ppf(1-a/2, 2*k + 2) / 2)
    if k == 0: 
        low = 0.0
    return k - low, high - k

y = np.linspace(0.1, 1, 50)  # Dependent variable

# Mean expected counts
x_mu = bump_forward_model(y, 
                    amp_s=50, mu_s=0.8, std_s=0.05,  # Signal params
                    amp_b=50, exp_b=-0.5)  # Background params

# Realized counts
x = np.random.poisson(x_mu)
x_err = np.array([poisson_interval(k) for k in x.T]).T

# Plot
plt.plot(y, x_mu, color='k', ls='--', label="Mean expected counts")
plt.errorbar(y, x, yerr=x_err, fmt='o', color='k', label="Realized counts")

plt.xlabel("$y$")
plt.ylabel("Counts")

plt.legend()

# plt.savefig("assets/x1.pdf")

## The explicit likelihood

In this case, we can write down a log-likelihood as a Poisson over the mean returned by the forward model.

In [ ]:
def log_like(theta, y, x):
    """ Log-likehood function for a Gaussian bump (amp_s, mu_s, std_s) on top of a power-law background (amp_b, exp_b).
    """
    amp_s, mu_s, std_s, amp_b, exp_b = theta
    mu = bump_forward_model(y, amp_s, mu_s, std_s, amp_b, exp_b)
    return poisson.logpmf(x, mu).sum()

Let's focus on just 2 parameters for simplicity, the signal amplitude and mean location. The likelihood in this case is:

In [ ]:
def log_like_sig(params, y, x):
    """ Log-likehood function for a Gaussian bump (amp_s, mu_s) on top of a fixed PL background.
    """
    amp_s, mu_s = params
    std_s, amp_b, exp_b = 0.05, 50, -0.5
    mu = bump_forward_model(y, amp_s, mu_s, std_s, amp_b, exp_b)
    return poisson.logpmf(x, mu).sum()

log_like_sig([50, 0.8], y, x)

Get a maximum-liklelihood estimate:

In [ ]:
# Initial guess for the parameters
initial_guess = [100., 0.1]

# Set up the minimizer_kwargs for the basinhopping algorithm
minimizer_kwargs = {"method": "L-BFGS-B", "bounds": ((0, 200), (0, 1))}

# Perform the optimization using basinhopping
opt = basinhopping(lambda thetas: -log_like_sig(thetas, y, x), initial_guess, minimizer_kwargs=minimizer_kwargs)

print("MLE parameters: {}; true parameters: {}".format(opt.x, (50, 0.8)))


And approximate posterior using `emcee`:

In [ ]:
def log_prior(thetas):
    """ Log-prior function for a Gaussian bump (amp_s, mu_s) on top of a fixed PL background.
    """
    amp_s, mu_s = thetas
    if 0 < amp_s < 200 and 0 < mu_s < 2:
        return 0
    else:
        return -np.inf
    
def log_post(thetas, y, x):
    """ Log-posterior function for a Gaussian bump (amp_s, mu_s) on top of a fixed PL background.
    """
    lp = log_prior(thetas)
    if not np.isfinite(lp):
        return -np.inf
    else:
        return lp + log_like_sig(thetas, y, x)
    
# Sampling with `emcee`
ndim, nwalkers = 2, 32
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_post, args=(y, x))

pos = opt.x + 1e-3 * np.random.randn(nwalkers, ndim)
sampler.run_mcmc(pos, 5000, progress=True);

In [ ]:
# Plot posterior samples
flat_samples = sampler.get_chain(discard=1000, flat=True)
corner.corner(flat_samples, labels=["amp_s", "mu_s"], truths=[50, 0.8], smooth=1.);

## The implicit likelihood

Now we will do inference without relying on the explicit likelihood evaluation. The key realization is that samples from the forward model implicitly encode the likelihood; when we are simulating data points $x$ for different parameter points $\theta$, we are drawing samples from the likelihood:
$$x\sim p(x\mid\theta)$$
which is where the _implicit_ aspect comes from. Let's write down a bump simulator:

In [ ]:
def bump_simulator(thetas, y):
    """ Simulate samples from the bump forward model given theta = (amp_s, mu_s) and abscissa points y.
    """
    amp_s, mu_s = thetas
    std_s, amp_b, exp_b = 0.05, 50, -0.5
    x_mu = bump_forward_model(y, amp_s, mu_s, std_s, amp_b, exp_b)
    x = np.random.poisson(x_mu)
    return x

# Test it out
bump_simulator([50, 0.8], y)

### Approximate Bayesian Computation (ABC)

The idea behind ABC is to realize samples from the forward model (with the parameters $\theta$ drawn from a prior) and compare it to the dataset of interest $x$. If the data and realized samples are close enough to each other according to some criterion, we keep the parameter points.

The comparison criterion here is a simple MSE on the data points. Play around with the parameters of the forward model to see how the criterion `eps` changes.

In [ ]:
x_fwd = bump_simulator([50, 0.8], y)
eps = np.sum(np.abs(x - x_fwd) ** 2) / len(x)
eps

In [ ]:
def abc(y, x, eps_thresh=500., n_samples=1000):
    """ABC algorithm for Gaussian bump model.

    Args:
        y (np.ndarray): Abscissa points.
        x (np.ndarray): Data counts.
        eps_thresh (float, optional): Acceptance threshold. Defaults to 500.0.
        n_samples (int, optional): Number of samples after which to stop. Defaults to 1000.

    Returns:
        np.ndarray: Accepted samples approximating the posterior p(theta|x).
    """
    samples = []
    total_attempts = 0
    progress_bar = tqdm(total=n_samples, desc="Accepted Samples", unit="samples")

    # Keep simulating until we have enough accepted samples
    while len(samples) < n_samples:
        params = np.random.uniform(low=[0, 0], high=[200, 1])  # Priors; theta ~ p(theta)
        x_fwd = bump_simulator(params, y)  # x ~ p(x|theta)
        eps = np.sum(np.abs(x - x_fwd) ** 2) / len(x)  # Distance metric; d(x, x_fwd)
        total_attempts += 1

        # If accepted, add to samples
        if eps < eps_thresh:
            samples.append(params)
            progress_bar.update(1)
            acceptance_ratio = len(samples) / total_attempts
            progress_bar.set_postfix(acceptance_ratio=f"{acceptance_ratio:.3f}")

    progress_bar.close()
    return np.array(samples)

n_samples = 5_000
post_samples = abc(y, x, eps_thresh=200, n_samples=n_samples)

In [ ]:
fig = corner.corner(post_samples, labels=["amp_s", "mu_s"], truths=[50, 0.8], range=[(0, 200), (0.3, 1)], bins=50);
corner.corner(flat_samples, labels=["amp_s", "mu_s"], truths=[50, 0.8], fig=fig, color="red", weights=np.ones(len(flat_samples)) * n_samples / len(flat_samples), range=[(0, 200), (0.3, 1)], bins=50);

Downsides of vanilla ABC:
- How to summarize the data? Curse of dimensionality / loss of information.
- How to compare with data? Likelihood may not be available.
- Choice of acceptance threshold: Precision/efficiency tradeoff.
- Need to re-run pipeline for new data or new prior.

### Neural likelihood-ratio estimation (NRE)

For numerical stability, the alternate hypothesis $\theta_0$ can be assumed to be one where $x$ and $\theta$ are not correlated, i.e., drawn from the individual marginal distributions $\{x, \theta\} \sim p(x)\,p(\theta)$. Then the alternate has support over the entire parameter space, instead of being a single hypothesis $\theta_0$.

In this case, we get the likelihood-to-evidence ratio,

$$\hat r(x, \theta) = \frac{s(x, \theta)}{1 - s(x, \theta)} = \frac{p(x,\theta)}{p(x)p(\theta)} = \frac{p(x\mid\theta)}{p(x)}$$

$\hat r(x, \theta)$ can be shown to be the classifier logit, i.e. the output before softmaxxing into the decision function with range between 0 and 1.

Start by creating some training data.

In [ ]:
n_train = 50_000

# Simulate training data
theta_samples = np.random.uniform(low=[0, 0], high=[200, 1], size=(n_train, 2))  # Parameter proposal
x_samples = np.array([bump_simulator(theta, y) for theta in tqdm(theta_samples)])

# Convert to torch tensors
theta_samples = torch.tensor(theta_samples, dtype=torch.float32)
x_samples = torch.tensor(x_samples, dtype=torch.float32)

# Normalize the data
x_mean = x_samples.mean(dim=0)
x_std = x_samples.std(dim=0)
x_samples = (x_samples - x_mean) / x_std

theta_mean = theta_samples.mean(dim=0)
theta_std = theta_samples.std(dim=0)
theta_samples = (theta_samples - theta_mean) / theta_std

As our parameterized classifier, we will use a simple MLP.

In [ ]:
def build_mlp(input_dim, hidden_dim, output_dim, layers, activation=nn.GELU()):
    """Create an MLP from the configuration."""
    seq = [nn.Linear(input_dim, hidden_dim), activation]
    for _ in range(layers):
        seq += [nn.Linear(hidden_dim, hidden_dim), activation]
    seq += [nn.Linear(hidden_dim, output_dim)]
    return nn.Sequential(*seq)

Create a neural ratio estimator class, with a corresponding loss function. The loss is a simple binary cross-entropy loss that discriminates between samples from the joint distribution $\{x, \theta\} \sim p(x\mid\theta)$ and those from a product of marginals $\{x, \theta\} \sim p(x)\,p(\theta)$. Samples from the latter are obtained by shuffling joint samples from within a batch.

The binary cross-entropy loss is used as the classifier loss to distinguish samples from the joint and marginals,
$$\mathcal L = - \sum_i y_i \log(p_i)$$
where $y_i$ are the true labels and $p_i$ the softmaxxed probabilities.

In [ ]:
class NeuralRatioEstimator(pl.LightningModule):
    """ Simple neural likelihood-to-evidence ratio estimator, using an MLP as a parameterized classifier.
    """
    def __init__(self, x_dim, theta_dim):
        super().__init__()
        self.classifier = build_mlp(input_dim=x_dim + theta_dim, hidden_dim=128, output_dim=1, layers=4)

    def forward(self, x):
        return self.classifier(x)
    
    def loss(self, x, theta):

        # Repeat x in groups of 2 along batch axis
        x = x.repeat_interleave(2, dim=0)

        # Get a shuffled version of theta
        theta_shuffled = theta[torch.randperm(theta.shape[0])]

        # Interleave theta and shuffled theta
        theta = torch.stack([theta, theta_shuffled], dim=1).reshape(-1, theta.shape[1])

        # Get labels; ones for pairs from joint, zeros for pairs from marginals
        labels = torch.ones(x.shape[0], device=x.device) 
        labels[1::2] = 0.0

        # Pass through parameterized classifier to get logits
        logits = self(torch.cat([x, theta], dim=1))
        probs = torch.sigmoid(logits).squeeze()

        return nn.BCELoss(reduction='none')(probs, labels)


    def training_step(self, batch, batch_idx):
        x, theta = batch
        loss = self.loss(x, theta).mean()
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, theta = batch
        loss = self.loss(x, theta).mean()
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=3e-4)

In [ ]:
# Evaluate loss; initially it should be around -log(0.5) = 0.693
nre = NeuralRatioEstimator(x_dim=50, theta_dim=2)
nre.loss(x_samples[:64], theta_samples[:64])

Instantiate dataloader and train.

In [ ]:

val_fraction = 0.1
batch_size = 128
n_samples_val = int(val_fraction * len(x_samples))

dataset = TensorDataset(x_samples, theta_samples)

dataset_train, dataset_val = random_split(dataset, [len(x_samples) - n_samples_val, n_samples_val])
train_loader = DataLoader(dataset_train, batch_size=batch_size, num_workers=8, pin_memory=True, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size, num_workers=8, pin_memory=True, shuffle=False)


In [ ]:
trainer = pl.Trainer(max_epochs=20)
trainer.fit(model=nre, train_dataloaders=train_loader, val_dataloaders=val_loader)

The classifier logits are now an estimator for the likelihood ratio. We can write down a log-likelihood function and use it to sample from the corresponding posterior distribution, just like before.

In [ ]:
def log_like(theta, x):
    """ Log-likelihood ratio estimator using trained classifier logits.
    """
        
    x = torch.Tensor(x)
    theta = torch.Tensor(theta)

    # Normalize
    x = (x - x_mean) / x_std
    theta = (theta - theta_mean) / theta_std

    x = torch.atleast_1d(x)
    theta = torch.atleast_1d(theta)

    return nre.classifier(torch.cat([x, theta], dim=-1)).squeeze()

theta_test = np.array([90, 0.8])
x_test = bump_simulator(theta_test, y)

log_like(theta_test, x_test)

In [ ]:
def log_post(theta, x):
    """ Log-posterior distribution, for sampling.
    """
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    else:
        return lp + log_like(theta, x)

Sample with `emcee`:

In [ ]:
ndim, nwalkers = 2, 32
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_post, args=(x_test,))

pos = opt.x + 1e-3 * np.random.randn(nwalkers, ndim)
sampler.run_mcmc(pos, 5000, progress=True);

Plot approximate posterior:

In [ ]:
flat_samples = sampler.get_chain(discard=1000, flat=True)
corner.corner(flat_samples, labels=["amp_s", "mu_s"], truths=[90, 0.8]);

### Neural posterior estimation (NPE)

In [ ]:
from nflows.flows.base import Flow
from nflows.distributions.normal import StandardNormal
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation

In [ ]:
def get_flow(d_in=2, d_hidden=32, d_context=16, n_layers=4):
    """ Instantiate a simple (Masked Autoregressive) normalizing flow.
    """

    base_dist = StandardNormal(shape=[d_in])

    transforms = []
    for _ in range(n_layers):
        transforms.append(ReversePermutation(features=d_in))
        transforms.append(MaskedAffineAutoregressiveTransform(features=d_in, hidden_features=d_hidden, context_features=d_context))
    transform = CompositeTransform(transforms)

    flow = Flow(transform, base_dist)
    return flow

# Instantiate flow
flow = get_flow()

# Make sure sampling and log-prob calculation makes sense
samples, log_prob = flow.sample_and_log_prob(num_samples=100, context=torch.randn(2, 16))
print(samples.shape, log_prob.shape)

Construct a neural posterior estimator. It uses a normalizing flow as a (conditional) posterior density estimator, and a feature-extraction network that aligns the directions of variations in parameters $\theta$ and data $x$.
$$  \mathcal L = -\log p_\phi(\theta\mid s_\varphi(x))$$
where $\{\phi, \varphi\}$ are the parameters of the normalizing flow and featurizer MLP, respectively.

In [ ]:
class NeuralPosteriorEstimator(pl.LightningModule):
    """ Simple neural posterior estimator class using a normalizing flow as the posterior density estimator.
    """
    def __init__(self, featurizer, d_context=16):
        super().__init__()
        self.featurizer = featurizer
        self.flow = get_flow(d_in=2, d_hidden=32, d_context=d_context, n_layers=4)

    def forward(self, x):
        return self.featurizer(x)
    
    def loss(self, x, theta):
        context = self(x)
        return -self.flow.log_prob(inputs=theta, context=context)

    def training_step(self, batch, batch_idx):
        x, theta = batch
        loss = self.loss(x, theta).mean()
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, theta = batch
        loss = self.loss(x, theta).mean()
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=3e-4)

Instantiate the NPE class and look at the loss:

In [ ]:
npe = NeuralPosteriorEstimator(featurizer=build_mlp(input_dim=50, hidden_dim=128, output_dim=16, layers=4))
npe.loss(x_samples[:64], theta_samples[:64])

Train using the same data as before:

In [ ]:
trainer = pl.Trainer(max_epochs=20)
trainer.fit(model=npe, train_dataloaders=train_loader, val_dataloaders=val_loader);

Get a test data sample, pass it through the feature extractor, and condition the flow density estimator on it. We get posterior samples by drawing from 
$$\theta \sim p_\phi(\theta\mid s_\varphi(x)).$$

In [ ]:
theta_test = np.array([90, 0.8])
x_test = bump_simulator(theta_test, y)

In [ ]:
x_test_norm = (torch.Tensor(x_test) - x_mean) / x_std
context = npe.featurizer(x_test_norm).unsqueeze(0)

In [ ]:
samples_test = npe.flow.sample(num_samples=10000, context=context) * theta_std + theta_mean
samples_test = samples_test.detach().numpy()
corner.corner(samples_test, labels=["amp_s", "mu_s"], truths=[90, 0.8]);

# A more complicated example: distribution of point sources in a 2D image

Finally, let's look at a more complicated example, one that is closer to a typical application of SBI and where the likelihood is formally intractable.

The forward model simulates a map of point sources with mean counts drawn from a power law (Pareto) distribution. The distribution of the mean counts is given by the following equation:

$$
\frac{\mathrm dn}{\mathrm  ds} = A s^{\beta}
$$

where $A$ is the amplitude (amp_b), $s$ is the flux, and $\beta$ is the exponent (exp_b). The fluxes are drawn from a truncated power law with minimum and maximum bounds, $s_\text{min}$ and $s_\text{max}$, respectively.

The number of sources is determined by integrating the power law distribution within the flux limits and taking a Poisson realization:

$$
N_\text{sources} \sim \text{Pois}\left(\int_{s_\text{min}}^{s_\text{max}} \, \mathrm ds \frac{\mathrm dn}{\mathrm ds}\right)
$$

For each source, a position is randomly assigned within the box of size `box_size`. The fluxes are then binned into a grid with `resolution` number of bins in both x and y directions. The resulting map is convolved with a Gaussian point spread function (PSF) with a standard deviation of `sigma_psf` to account for the spatial resolution of the instrument.

The output is a 2D map of counts, representing the simulated observation of the point sources in the sky.

In [ ]:
from scipy.stats import binned_statistic_2d
from astropy.convolution import convolve, Gaussian2DKernel


def simulate_sources(amp_b, exp_b, s_min=0.5, s_max=50.0, box_size=1., resolution=64, sigma_psf=0.01):
    """ Simulate a map of point sources with mean counts drawn from a power law (Pareto) distribution dn/ds = amp_b * s ** exp_b
    """
    # Get number of sources by analytically integrating dn/ds and taking Poisson realization
    n_sources = np.random.poisson(-amp_b * (s_min ** (exp_b - 1)) / (exp_b - 1))

    # Draw fluxes from truncated power law amp_b * s ** (exp_b - 1), with s_min and s_max as the bounds
    fluxes = draw_powerlaw_flux(n_sources, s_min, s_max, exp_b)

    positions = np.random.uniform(0., box_size, size=(n_sources, 2))
    bins = np.linspace(0, box_size, resolution + 1)

    pixel_size = box_size / resolution
    kernel = Gaussian2DKernel(x_stddev=1.0 * sigma_psf / pixel_size)

    mu_signal = binned_statistic_2d(x=positions[:, 0], y=positions[:, 1], values=fluxes, statistic='sum', bins=bins).statistic
    counts = np.random.poisson(convolve(mu_signal, kernel))
                
    return fluxes, counts

def draw_powerlaw_flux(n_sources, s_min, s_max, exp_b):
    """
    Draw from a powerlaw with slope `exp_b` and min/max mean counts `s_min` and `s_max`. From:
    https://stackoverflow.com/questions/31114330/python-generating-random-numbers-from-a-power-law-distribution
    """
    u = np.random.uniform(0, 1, size=n_sources)
    s_low_u, s_high_u = s_min ** (exp_b + 1), s_max ** (exp_b + 1)
    return (s_low_u + (s_high_u - s_low_u) * u) ** (1.0 / (exp_b + 1.0))

fluxes, counts = simulate_sources(amp_b=200., exp_b=-1.2)
plt.imshow(counts, cmap='viridis', vmax=20)
plt.xlabel("Pixels")
plt.ylabel("Pixels")

In [ ]:
# Draw parameters from the prior
n_params = 16

amp_b_prior = (100., 300.)
exp_b_prior = (-2.0, -0.5)

amp_bs = np.random.uniform(amp_b_prior[0], amp_b_prior[1], n_params)
exp_bs = np.random.uniform(exp_b_prior[0], exp_b_prior[1], n_params)

# Plot the data samples on a grid
fig, axes = plt.subplots(4, 4, figsize=(12, 12))

for i, ax in enumerate(axes.flatten()):
    fluxes, counts = simulate_sources(amp_b=amp_bs[i], exp_b=exp_bs[i])
    im = ax.imshow(counts, cmap='viridis', vmax=20)
    ax.set_title(f'$A_b={amp_bs[i]:.2f}, n_b={exp_bs[i]:.2f}$')
    ax.set_xticks([])
    ax.set_yticks([])

plt.show()

## Explicit likelihood

The (marginal) likelihood, which we would need to plug into something like MCMC, is computationally intractable! This is because it involves an integral over a cumbersome latent space, which consists of all possible number $n$ of sources and their positions $\{z\}=\{x, y\}_{i=1}^{n}$. Let's write this out formally:
$$p(x \mid \theta)=\sum_{n} \int \mathrm{d}^{n} \{z\}\, p\left(n \mid \theta\right) \prod_i^{n} p\left(z_{i} \mid \theta\right) \, p\left(x \mid \theta,\left\{z_{i}\right\}\right)$$

## Implicit inference: Neural posterior estimation

Let's use neural posterior estimation with a normalizing flow again. Get a training sample:

In [ ]:
n_train = 30_000

# Sample from prior, then simulate
theta_samples = np.random.uniform(low=[10., -3.], high=[200., -0.99], size=(n_train, 2))
x_samples = np.array([simulate_sources(theta[0], theta[1])[1] for theta in tqdm(theta_samples)])

# Convert to torch tensors
theta_samples = torch.Tensor(theta_samples)
x_samples = torch.Tensor(x_samples)

# Normalize the data
x_mean = x_samples.mean(dim=0)
x_std = x_samples.std(dim=0)
x_samples = (x_samples - x_mean) / x_std

theta_mean = theta_samples.mean(dim=0)
theta_std = theta_samples.std(dim=0)
theta_samples = (theta_samples - theta_mean) / theta_std


In [ ]:
val_fraction = 0.1
batch_size = 128
n_samples_val = int(val_fraction * len(x_samples))

dataset = TensorDataset(x_samples, theta_samples)

dataset_train, dataset_val = random_split(dataset, [len(x_samples) - n_samples_val, n_samples_val])
train_loader = DataLoader(dataset_train, batch_size=batch_size, num_workers=8, pin_memory=True, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size, num_workers=8, pin_memory=True, shuffle=False)


Since we're working with images, use a simple convolutional neural network (CNN) as the feature extractor. The normalizing flow will be conditioned on the output of the CNN.

In [ ]:
class CNN(nn.Module):
    """ Simple CNN feature extractor.
    """
    def __init__(self, output_dim):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(16 * 16 * 16, 64)
        self.fc2 = nn.Linear(64, output_dim)

    def forward(self, x):
        
        x = x.unsqueeze(1)  # Add channel dim
        
        x = self.pool1(F.leaky_relu(self.conv1(x), negative_slope=0.02))
        x = self.pool2(F.leaky_relu(self.conv2(x), negative_slope=0.02))

        x = x.view(x.size(0), -1)

        x = F.leaky_relu(self.fc1(x), negative_slope=0.01)
        x = self.fc2(x)

        return x

In [ ]:
npe = NeuralPosteriorEstimator(featurizer=CNN(output_dim=32), d_context=32)
npe.loss(x_samples[:64], theta_samples[:64])

In [ ]:
trainer = pl.Trainer(max_epochs=15)
trainer.fit(model=npe, train_dataloaders=train_loader, val_dataloaders=val_loader);

In [ ]:
npe = npe.eval()

Get a test map, extract features, condition normalizing flow, extract samples.

In [ ]:
params_test = np.array([15., -1.4])
x_test = simulate_sources(params_test[0], params_test[1])[1]

In [ ]:
x_test_norm = (torch.Tensor(x_test) - x_mean) / x_std
context = npe.featurizer(x_test_norm.unsqueeze(0))

samples_test = npe.flow.sample(num_samples=10000, context=context) * theta_std + theta_mean
samples_test = samples_test.detach().numpy()

corner.corner(samples_test, labels=["amp", "exp"], truths=params_test);

## Test of statistical coverage

<img src=./assets/coverage.png alt= “” width=800>

Figure from 2209.01845.

We can do some checks to make sure that our posterior has the correct statistical interpretation. In particular, let's test the posterior statistical coverage by evaluating how well the Highest Posterior Density (HPD) intervals capture the true parameter values.

The **Highest Posterior Density (HPD)** interval is a region in the parameter space that contains the most probable values for a given credible mass (e.g., 95% or 99%). In other words, it is the shortest interval that contains the specified credible mass of the posterior distribution. This is one of summarizing a posterior distribution.

**Nominal coverage** is the probability, or the proportion of the parameter space, that the HPD interval is intended to contain. For example, if the nominal coverage is 0.95, the HPD interval should theoretically contain the true parameter value 95% of the time.

**Empirical coverage** is the proportion of true parameter values that actually fall within the HPD interval, based on a set of test cases or simulations.

For a perfectly calibrated posterior estimator, empirical coverage = nominal coverage for all credibility levels.

In [ ]:
n_test = 200  # How many test samples to draw for coverage test

# Get samples 
x_test = torch.Tensor([simulate_sources(params_test[0], params_test[1])[1] for _ in range(n_test)])
x_test_norm = (torch.Tensor(x_test) - x_mean) / x_std

# and featurize
context = npe.featurizer(x_test_norm)

# Get posterior for all samples together in a batch
samples_test = npe.flow.sample(num_samples=1000, context=context) * theta_std + theta_mean
samples_test = samples_test.detach().numpy()

In [ ]:
def hpd(samples, credible_mass=0.95):
    """Compute highest posterior density (HPD) of array for given credible mass."""
    sorted_samples = np.sort(samples)
    interval_idx_inc = int(np.floor(credible_mass * sorted_samples.shape[0]))
    n_intervals = sorted_samples.shape[0] - interval_idx_inc
    interval_width = np.zeros(n_intervals)
    for i in range(n_intervals):
        interval_width[i] = sorted_samples[i + interval_idx_inc] - sorted_samples[i]
    hdi_min = sorted_samples[np.argmin(interval_width)]
    hdi_max = sorted_samples[np.argmin(interval_width) + interval_idx_inc]
    return hdi_min, hdi_max

hpd(samples_test[0, :, 0], credible_mass=0.2)

In [ ]:
p_nominals = np.linspace(0.01, 0.99, 50)
contains_true = np.zeros((2, n_test, len(p_nominals)))

for i_param in range(2):
    for i, sample in enumerate(samples_test[:, :, i_param]):
        for j, p_nominal in enumerate(p_nominals):
            hdi_min, hdi_max = hpd(sample, credible_mass=p_nominal)
            if hdi_min < params_test[i_param] < hdi_max:
                contains_true[i_param, i, j] = 1

In [ ]:
# Make two plots, one for each parameter

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].plot(p_nominals, contains_true[0].sum(0) / n_test)
ax[0].plot([0, 1], [0, 1], color="black", linestyle="--")
ax[0].set_xlabel("Nominal coverage")
ax[0].set_ylabel("Expected coverage")
ax[0].set_title("Coverage for amplitude")



ax[1].plot(p_nominals, contains_true[1].sum(0) / n_test)
ax[1].plot([0, 1], [0, 1], color="black", linestyle="--")
ax[1].set_xlabel("Nominal coverage")
ax[1].set_ylabel("Expected coverage")
ax[1].set_title("Coverage for exponent")

# In-class exercise: Damped Harmonic Oscillator

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device=}")

\begin{align}
\ddot{x}(t) + 2\beta\omega_0\dot{x}(t) + \omega_0^2x(t) = 0
\end{align}

Ansatz, $x = \exp(\gamma t)$; $\gamma = -\omega_0\left[\beta \pm i\sqrt{1 - \beta^2}\right]$

In [ ]:
def damped_sho(t, omega_0, beta, shift=0):
    # beta less than 1 for underdamped
    envel = beta * omega_0
    osc = torch.sqrt(1 - beta**2) * omega_0
    tau = t - shift
    data = torch.exp(-envel * tau) * torch.cos(osc * tau)
    data[tau < 0] = 0  # assume oscillator starts at tau = 0
    return data

def damped_sho_bilby(t, omega_0, beta, shift=0):
    # beta less than 1 for underdamped
    envel = beta * omega_0
    osc = np.sqrt(1 - beta**2) * omega_0
    tau = t - shift
    data = np.exp(-envel * tau) * np.cos(osc * tau)
    data[tau < 0] = 0  # assume oscillator starts at tau = 0

    return data

## Plot

In [ ]:
omega_0 = torch.tensor(1.94)
beta = torch.tensor(0.4)

t_vals = torch.linspace(-1, 10, 200)
for shift in [-1, 0, 1, 2]:
    x_vals = damped_sho(t_vals, omega_0=omega_0, beta=beta, shift=shift)
    plt.plot(t_vals, x_vals, label=f"{omega_0=}; {beta=}; {shift=}")
plt.xlabel("Time")
plt.ylabel("Disp")
plt.axvline(x=0, linestyle='dotted', color='black')
plt.legend()

In [ ]:
injection_parameters = dict(omega_0=omega_0, beta=beta, shift=2)
print(injection_parameters)

In [ ]:
num_points = 200
t_vals = np.linspace(-1, 10, num_points)

In [ ]:
sigma = 0.4

In [ ]:
data = damped_sho(t_vals, **injection_parameters) + np.random.normal(0, sigma, t_vals.size)

In [ ]:
fig, ax = plt.subplots()
ax.plot(t_vals, data, 'o', label='$\\{x_i, y_i\\}$')
ax.plot(t_vals, damped_sho(t_vals, **injection_parameters), '--r', label='f(x)')
ax.legend()
ax.set_xlabel('x')
ax.set_ylabel('y')

In [ ]:
import bilby
from bilby.core.prior import Uniform, DeltaFunction

## Prior for parameters

In [ ]:
priors = dict()

priors['omega_0'] = Uniform(0.1, 2, name='omega_0', latex_label='$\omega_0$') #np array
priors['beta'] = Uniform(0, 0.5, name='beta', latex_label='$\\beta$') #np array
priors['shift'] = Uniform(-4, 4, name='shift', latex_label='$\Delta\;t$')

In [ ]:
from IPython.display import clear_output
from time import sleep

In [ ]:
import torch

def get_data(omega_0=None, beta=None, shift=None, num_points=1):
    """Sample omega, beta, shift and return a batch of data with noise"""
    omega_0 = priors['omega_0'].sample() if omega_0 is None else omega_0
    omega_0 = torch.tensor(omega_0).to(dtype=torch.float32)
    beta = priors['beta'].sample() if beta is None else beta
    beta = torch.tensor(beta).to(dtype=torch.float32)
    shift = priors['shift'].sample() if shift is None else shift
    shift = torch.tensor(shift).to(dtype=torch.float32)

    t_vals = torch.linspace(-1, 10, num_points).to(dtype=torch.float32) #

    y = damped_sho(t_vals, omega_0=omega_0, beta=beta, shift=shift)
    y += sigma * torch.randn(size=y.size()).to(dtype=torch.float32)

    return t_vals, y, omega_0, beta, shift

# Add augmentation

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch import nn, optim
# from torch.utils.tensorboard import SummaryWriter
from tensorboardX import SummaryWriter

In [ ]:
%%time

num_simulations = 100000
num_repeats = 10

theta_unshifted_vals = []
theta_shifted_vals = []
data_unshifted_vals = []
data_shifted_vals = []

for ii in range(num_simulations):
    # generated data with a fixed shift
    t_vals, y_unshifted, omega, beta, shift = get_data(num_points=num_points, shift=1)
    # create repeats
    theta_unshifted = torch.tensor([omega, beta, shift]).repeat(num_repeats, 1).to(device=device)
    theta_unshifted_vals.append(theta_unshifted)
    data_unshifted_vals.append(y_unshifted.repeat(num_repeats, 1).to(device=device))
    # generate shifted data
    theta_shifted = []
    data_shifted = []
    for _ in range(num_repeats):
        t_val, y_shifted, _omega, _beta, shift = get_data(
            omega_0=omega, beta=beta,  # omega and beta same as above
            shift=None,
            num_points=num_points
        )
        theta_shifted.append(torch.tensor([omega, beta, shift]))
        data_shifted.append(y_shifted)
    theta_shifted_vals.append(torch.stack(theta_shifted).to(device=device))
    data_shifted_vals.append(torch.stack(data_shifted).to(device=device))

In [ ]:
class DataGenerator(Dataset):
    def __len__(self):
        return num_simulations

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return (
            theta_shifted_vals[idx].to(dtype=torch.float32),
            theta_unshifted_vals[idx].to(dtype=torch.float32),
            data_shifted_vals[idx].to(dtype=torch.float32),
            data_unshifted_vals[idx].to(dtype=torch.float32)
        )

In [ ]:
dataset = DataGenerator()

In [ ]:
_, t, d, _ = dataset[6]

fig, ax = plt.subplots()

for (omega, beta, shift), points in zip(t, d):
    ax.plot(t_vals.clone().detach().cpu().numpy(), points.clone().detach().cpu().numpy(),
            'o', markersize=0.8)
ax.set_xlabel('t')
ax.set_ylabel('y')
ax.set_title(f"Augmented sample; $\\omega_0$ = {t[0][0]:.1f}; $\\beta$ = {t[0][1]:.1f}")

In [ ]:
train_set_size = int(0.8 * num_simulations)
val_set_size = int(0.1 * num_simulations)
test_set_size = int(0.1 * num_simulations)

train_data, val_data, test_data = torch.utils.data.random_split(
    dataset, [train_set_size, val_set_size, test_set_size])

In [ ]:
TRAIN_BATCH_SIZE = 1000
VAL_BATCH_SIZE = 1000

train_data_loader = DataLoader(
    train_data, batch_size=TRAIN_BATCH_SIZE,
    shuffle=True
)

val_data_loader = DataLoader(
    val_data, batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_data_loader = DataLoader(
    test_data, batch_size=1,
    shuffle=False
)

In [ ]:
for theta, _, data_aug, data_orig in train_data_loader:
    break

In [ ]:
len(train_data_loader), len(val_data_loader), len(test_data_loader)

In [ ]:
theta.shape, data_aug.shape, data_orig.shape

In [ ]:
len(val_data_loader)

#  No Embedding Net used

In [ ]:
from nflows.distributions import StandardNormal
from nflows.flows import Flow
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform
from nflows.transforms import CompositeTransform, RandomPermutation

import nflows.utils as torchutils

In [ ]:
num_transforms = 5
num_blocks = 4
hidden_features = 50

context_features = num_points

base_dist = StandardNormal([2])

transforms = []
for _ in range(num_transforms):
    block = [
        MaskedAffineAutoregressiveTransform(
            features=2,  # 2-dim posterior
            hidden_features=hidden_features,
            context_features=context_features,
            num_blocks=num_blocks,
            activation=torch.tanh,
            use_batch_norm=False,
            use_residual_blocks=True,
            dropout_probability=0.01,
        ),
        RandomPermutation(features=2)
    ]
    transforms += block

transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist).to(device)

In [ ]:
print("Total number of trainable parameters: ", sum(p.numel() for p in flow.parameters() if p.requires_grad))

# Train/Validate

In [ ]:
num_augmentations = 10

def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for idx, val in enumerate(train_data_loader, 1):
        augmented_theta, _, augmented_data, _ = val
        augmented_theta = augmented_theta[...,0:2]

        loss = 0
        for ii in range(num_augmentations):
            theta = augmented_theta[:,ii,:]
            data = augmented_data[:,ii,:]

            flow_loss = -flow.log_prob(theta, context=data).mean()

            optimizer.zero_grad()
            flow_loss.backward()
            optimizer.step()
            loss += flow_loss.item()

        running_loss += loss/num_augmentations
        if idx % 10 == 0:
            last_loss = running_loss / 10 # avg loss
            print(' Avg. train loss/batch after {} batches = {:.4f}'.format(idx, last_loss))
            tb_x = epoch_index * len(train_data_loader) + idx
            tb_writer.add_scalar('Flow Loss/train', last_loss, tb_x)
            tb_writer.flush()
            running_loss = 0.
    return last_loss


def val_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for idx, val in enumerate(val_data_loader, 1):
        augmented_theta, _, augmented_data, _ = val
        augmented_theta = augmented_theta[...,0:2]

        loss = 0
        for ii in range(num_augmentations):
            theta = augmented_theta[:,ii,:]
            data = augmented_data[:,ii,:]

            flow_loss = -flow.log_prob(theta, context=data).mean()
            loss += flow_loss.item()

        running_loss += loss/num_augmentations
        if idx % 5 == 0:
            last_loss = running_loss / 5
            tb_x = epoch_index * len(val_data_loader) + idx + 1
            tb_writer.add_scalar('Flow Loss/val', last_loss, tb_x)

            tb_writer.flush()
            running_loss = 0.
    tb_writer.flush()
    return last_loss

In [ ]:
optimizer = optim.SGD(flow.parameters(), lr=1e-3, momentum=0.5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, threshold=0.01)

In [ ]:
writer = SummaryWriter("damped-harmonic-oscillator-baseline.neurips", comment="With LR=1e-3", flush_secs=5)
# epoch_number = 0

In [ ]:
PATH2 = './damped-harmonic-oscillator-baseline.neurips.pth'
flow.load_state_dict(torch.load(PATH2, map_location=device))
flow.eval()

In [ ]:
# %%time
# # UNCOMMENT AND RUN TO TRAIN FROM SCRATCH
EPOCHS = 30

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))
    # Gradient tracking
    flow.train(True)
    # flow._embedding_net.train(False)
    # no gradient tracking for embedding layer
    for name, param in flow._embedding_net.named_parameters():
        param.requires_grad = False

    avg_train_loss = train_one_epoch(epoch, writer)

    # no gradient tracking, for validation
    flow.train(False)
    avg_val_loss = val_one_epoch(epoch, writer)

    print(f"Train/Val flow Loss after epoch: {avg_train_loss:.4f}/{avg_val_loss:.4f}")

    scheduler.step(avg_val_loss)

In [ ]:
# PATH2 = './damped-harmonic-oscillator-baseline.notebook.pth'
# torch.save(flow.state_dict(), PATH2)

# Check on test data

In [ ]:
import pandas as pd
import corner

def cast_as_bilby_result(samples, truth):
    injections = dict.fromkeys({'omega_0', 'beta'})
    injections['omega_0'] = float(truth.numpy()[0])
    injections['beta'] = float(truth.numpy()[1])

    posterior = dict.fromkeys({'omega_0', 'beta'})
    samples_numpy = samples.numpy()
    posterior['omega_0'] = samples_numpy.T[0].flatten()
    posterior['beta'] = samples_numpy.T[1].flatten()
    posterior = pd.DataFrame(posterior)

    return bilby.result.Result(
        label="test_data",
        injection_parameters=injections,
        posterior=posterior,
        search_parameter_keys=list(injections.keys()),
        priors=priors
    )

In [ ]:
def live_plot_samples(samples, truth):
    print(truth)
    clear_output(wait=True)
    sleep(0.5)
    figure = corner.corner(
        samples.numpy(), quantiles=[0.05, 0.5, 0.95],
        show_titles=True,
        labels=["omega_0", "beta",],
        truth=truth,
    )

    corner.overplot_lines(figure, truth, color="C1")
    corner.overplot_points(figure, truth[None], marker="s", color="C1")

## Posteriors

In [ ]:
for idx, (_, theta_test, data_test, data_orig) in enumerate(test_data_loader):
    if idx % 1000 !=0: continue 
    with torch.no_grad():
        samples = flow.sample(3000, context=data_test[0][0].reshape((1, 200)))
    live_plot_samples(samples.cpu().reshape(3000,2), theta_test[0][0].cpu()[...,0:2])
    plt.show()

# PP plot

In [ ]:
results = []
for idx, (_, theta_test, data_test, data_unshifted) in enumerate(test_data_loader):
    if idx == 1000: break  # full set takes time
    with torch.no_grad():
        samples = flow.sample(3000, context=data_test[0][0].reshape((1, 200)))
    results.append(
        cast_as_bilby_result(samples.cpu().reshape(3000, 2),
                             theta_test[0][0].cpu()[...,0:2]))

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    bilby.result.make_pp_plot(results, save=False, keys=['omega_0', 'beta'])

# Comparison with bilby and analytic result

In [ ]:
import bilby
bilby_result = bilby.result.Result.from_json('./bilby_sho.json', outdir=None)

In [ ]:
bilby_result.injection_parameters.pop('shift')
injection_parameters = bilby_result.injection_parameters.copy()

In [ ]:
injection_parameters

In [ ]:
flow_sim_result = bilby.result.Result.from_json(filename='./sim_flow_sho.json', outdir=None)

In [ ]:
x_vals = damped_sho_bilby(t_vals, **injection_parameters)

In [ ]:
try:
    x_vals = torch.from_numpy(x_vals).to(device, dtype=torch.float32)
except TypeError:
    x_vals = x_vals.to(device, dtype=torch.float32)

In [ ]:
with torch.no_grad():
    flow_samples = flow.sample(3000, context=x_vals.reshape((1, 200)))

In [ ]:
flow_result = cast_as_bilby_result(flow_samples.cpu().reshape(3000, 2),
                                   torch.tensor(list(injection_parameters.values())))

In [ ]:
fig = bilby.result.plot_multiple(
    [bilby_result, flow_sim_result, flow_result],
    labels=['Nested Samp.', 'Flow with Rep. (99K params)', 'Flow Baseline. (355K params)'],
    truth=injection_parameters,
    quantiles=(0.16, 0.84),
    titles=True, save=True,
    filename='sho-comparison-bilby-with-sim.pdf'
)

# In-class exercise: Linear Regression

In [ ]:
def linear_model(x, m, c):
    return m*x + c

### Putting some numbers

In [ ]:
injection_parameters = dict(m=0.8, c=2)

In [ ]:
num_points = 50
x = np.linspace(-4, 4, num_points)

In [ ]:
sigma = 0.6

In [ ]:
data = linear_model(x, **injection_parameters) + np.random.normal(0, sigma, x.size)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, data, 'o', label='$\\{x_i, y_i\\}$')
ax.plot(x, linear_model(x, **injection_parameters), '--r', label='f(x)')
ax.legend()
ax.set_xlabel('x')
ax.set_ylabel('y')

In [ ]:
m_hat = (x * data).sum() / (x * x).sum()
c_hat = data.sum() / num_points

delta_m = sigma/np.sqrt(np.sum((x)**2))
delta_c = sigma * np.sqrt(1/num_points)

print("Expected m = {:.3f} +/- {:.3f}".format(m_hat, delta_m))

In [ ]:
print("Expected c = {:.3f} +/- {:.3f}".format(c_hat, delta_c))

In [ ]:
import bilby
from bilby.core.likelihood import GaussianLikelihood
from bilby.core.prior import Uniform, DeltaFunction

In [ ]:
priors = dict()

priors['m'] = Uniform(-3, 3, name='m', latex_label='m')
priors['c'] = Uniform(-3, 3, name='c', latex_label='c')

In [ ]:
log_l = GaussianLikelihood(x, data, linear_model, sigma=sigma)

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    result = bilby.run_sampler(
        likelihood=log_l, priors=priors, sampler='dynesty',
        nlive=500, npool=4, save=False, clean=True,
        injection_parameters=injection_parameters,
        outdir='./linear_regression',
        label='linear_regression'
    )

## Result from nested sampling

In [ ]:
result.plot_corner(priors=True, quantiles=(0.16, 0.84))

# Same problem using likelihood-free inference
By posterior estimation using a normalizing flow

- We can simulate signals
  - Given $\mathbf{\Theta}_i \sim p(\mathbf{\Theta}) \rightarrow$ generate signal $f(\mathbf{\Theta}_i)$
  - Add instrument noise $f(\mathbf{\Theta}_i) + n \rightarrow \mathbf{d}_i$
- We can "easily" get pairs $\{\mathbf{\Theta}_i, \mathbf{d}_i\}$
- From $\{\mathbf{\Theta}_i, \mathbf{d}_i\} \rightarrow p(\mathbf{\Theta}, \mathbf{d}), p(\mathbf{\Theta}\vert \mathbf{d}), p(\mathbf{d}\vert\mathbf{\Theta})$
- Here, we are interested in the posterior estimation

- The distributions are complex
- A solution is to use a Normalizing flow
  - Contains a **learnable transform** (a trainable neural network)
  - A **base distribution** (often taken to be normal)
- Here we use a affine-autogressive transform from `pyro`
- A standard normal base distribution

In [ ]:
from IPython.display import clear_output
from time import sleep

def live_plot_samples(samples, truth):
    clear_output(wait=True)
    sleep(1)
    figure = corner.corner(
        samples.numpy(), quantiles=[0.16, 0.5, 0.84],
        show_titles=True, labels=["m", "c"],
        truth=truth
    )

    corner.overplot_lines(figure, truth, color="C1")
    corner.overplot_points(figure, truth[None], marker="s", color="C1")


def live_plot_bilby_result(result, **kwargs):
    clear_output(wait=True)
    sleep(1)
    result.plot_corner(priors=True)

In [ ]:
def get_data(m=None, c=None, num_points=1):
    """Sample m, c and return a batch of data with noise"""
    m = priors['m'].sample() if m is None else m
    c = priors['c'].sample() if c is None else c
    x = np.linspace(-4, 4, num_points)
    y = m*x + c
    y += sigma*np.random.normal(size=x.size)

    return x, y, m, c

# Generate simulations

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
from lightning import pytorch as pl

In [ ]:
num_simulations = 40000
theta_vals = []
data_vals = []
for ii in range(num_simulations):
    x_val, y_val, m_val, c_val = get_data(num_points=num_points)
    data_vals.append(y_val)
    theta_vals.append([m_val, c_val])

In [ ]:
theta_vals = torch.from_numpy(np.array(theta_vals)).to(torch.float32)
data_vals = torch.from_numpy(np.array(data_vals)).to(torch.float32)

In [ ]:
class DataGenerator(Dataset):
    def __len__(self):
        return num_simulations

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return theta_vals[idx], data_vals[idx]

In [ ]:
dataset = DataGenerator()

In [ ]:
train_set_size = int(0.8 * num_simulations)
val_set_size = int(0.1 * num_simulations)
test_set_size = int(0.1 * num_simulations)

train_data, val_data, test_data = torch.utils.data.random_split(
    dataset, [train_set_size, val_set_size, test_set_size])

In [ ]:
TRAIN_BATCH_SIZE = 40
TEST_BATCH_SIZE = 1
LR = 1e-3

In [ ]:
import torch.distributions as dist

In [ ]:
from pyro.nn import ConditionalAutoRegressiveNN
from pyro.distributions import ConditionalTransformedDistribution
from pyro.distributions.transforms import ConditionalAffineAutoregressive

# MADE in Pytorch Lightning

Quick recap of terminology
- **Dataset**: the entire dataset, tuples of $\{\mathbf{\Theta}_i, \mathbf{d}_i\}$
- **Dataloader**: partition of dataset into batches i.e. gives a batch of data
- **Training loop**: One epoch
  - Push a batch of data
  - Calculate loss
  - Compute all gradients
  - Change all weights based on gradients
  - Repeats until all batches are done

Pytorch-lightning removes boilerplate code
- `torch.nn.Module` $\rightarrow$ `pl.LightningModule`
- Provides methods: `training_step`, `validation_step`, `test_step`, `configure_optimizers`
- No need of explicit training loop
- Automatic checkpoints, several options for logging
- Easily scales to multi-GPU distributed training

In [ ]:
import pandas as pd
import corner

def cast_as_bilby_result(samples, truth):
    injections = dict.fromkeys(injection_parameters)
    injections['m'] = float(truth.numpy()[0])
    injections['c'] = float(truth.numpy()[1])

    posterior = dict.fromkeys(injection_parameters)
    samples_numpy = samples.numpy()
    posterior['m'] = samples_numpy.T[0].flatten()
    posterior['c'] = samples_numpy.T[1].flatten()
    posterior = pd.DataFrame(posterior)
    
    return bilby.result.Result(
        label="test_data",
        injection_parameters=injections,
        posterior=posterior,
        search_parameter_keys=list(injections.keys()),
        priors=priors
    )

In [ ]:
class MADE(pl.LightningModule):
    def __init__(
        self,
        input_dim,
        context_dim,
        hidden_dim,
        learning_rate: float = LR,
        batch_size: int = TRAIN_BATCH_SIZE,
    ):
        super().__init__()
        self.base_dist = dist.Normal(torch.zeros(input_dim), torch.ones(input_dim))
        self.hypernet = ConditionalAutoRegressiveNN(input_dim, context_dim, hidden_dims)
        self.transform = ConditionalAffineAutoregressive(self.hypernet)
        self.flow = ConditionalTransformedDistribution(self.base_dist, [self.transform])

        self.learning_rate = learning_rate
        self.batch_size = batch_size

    def log_prob(self, theta, data):
        return self.flow.condition(data).log_prob(theta).mean()

    
    def training_step(self, batch, batch_idx):
        theta, data = batch
        loss = - self.log_prob(theta, data)
        self.log("train_loss", loss, on_step=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        theta, data = batch
        loss = - self.log_prob(theta, data)
        self.log("valid_loss", loss, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        theta, data = batch
        samples = self.flow.condition(data).sample([2000])
        res = cast_as_bilby_result(samples, theta[0])
        self.test_results.append(res)

    def configure_optimizers(self):
        parameters = self.transform.parameters()
        optimizer = torch.optim.AdamW(parameters, self.learning_rate)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            self.learning_rate,
            pct_start=0.1,
            total_steps=self.trainer.estimated_stepping_batches
        )
        scheduler_config = dict(scheduler=scheduler, interval="step")
        return dict(optimizer=optimizer, lr_scheduler=scheduler_config)
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            train_data,
            batch_size=self.batch_size,
            shuffle=True,
            pin_memory=True
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            val_data,
            batch_size=self.batch_size,
            shuffle=False,
            pin_memory=True
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            test_data,
            batch_size=1,
            shuffle=False,
            pin_memory=True
        )

In [ ]:
class PPPlotCallback(pl.Callback):
    def on_test_start(self, trainer, pl_module):
        pl_module.test_results = []

    def on_test_end(self, trainer, pl_module):
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            bilby.result.make_pp_plot(pl_module.test_results, save=False, keys=['m', 'c'])
        pl_module.test_results.clear()

In [ ]:
input_dim = 2
context_dim = num_points
hidden_dims = [5*input_dim, 5*input_dim]

model = MADE(input_dim, context_dim, hidden_dims, batch_size=40, learning_rate=5e-3)

trainer = pl.Trainer(
    max_epochs=20,
    log_every_n_steps=100,
    logger=pl.loggers.CSVLogger("logs", name="made-expt"),
    callbacks=[PPPlotCallback()]
)

In [ ]:
%%capture
trainer.fit(model)

# Example posteriors

In [ ]:
for idx, (theta_test, data_test) in enumerate(test_data):
    if idx == 5: break 
    with torch.no_grad():
        samples = model.flow.condition(data_test).sample([1000])
    live_plot_samples(samples, theta_test)
    plt.show()

In [ ]:
trainer.test(model)

## Result from normalizing flow

In [ ]:
with torch.no_grad():
    flow_samples = model.flow.condition(
        torch.from_numpy(data).unsqueeze(0).to(dtype=torch.float32)
    ).sample([1000])
truth = np.array(list(injection_parameters.values()))

In [ ]:
live_plot_samples(flow_samples, truth)